## I love to debug

In [186]:
import functools
import os
import time
from typing import List
import matplotlib.pyplot as plt
import cirq
import networkx as nx
import numpy as np
import qiskit
from mitiq import benchmarks, pec

from qiskit_ibm_runtime.fake_provider import FakeKolkata, FakeLima
from qiskit.providers.fake_provider import GenericBackendV2
import qiskit_aer

In [187]:
ideal_backend = qiskit_aer.AerSimulator()

# Random seed for circuit generation.
seed: int = 1

# Display verbose output.
verbose: bool = False
# Give queue updates every this many seconds when running on hardware device.
verbose_update_time: int = 30

In [188]:
def get_phys_qubits(n_qubits):
    # Physical qubits with a chain-like connectivity.
    ibm_lima_ordering = [0, 1, 3, 4, 2] 
    ibm_kolkata_ordering = [0, 1, 4, 7, 10, 12, 15, 18, 21, 23, 24, 25, 22, 19, 16, 14, 11, 8, 5, 3, 2]

    if n_qubits <= 5:
        # Assume ibmq-lima device. Take n adjacent physical qubits.
        physical_ibm_qubits = ibm_lima_ordering[: n_qubits]
    elif n_qubits <= len(ibm_kolkata_ordering) and n_qubits > 5:
        # Assume ibmq-kolkata device. Take n adjacent physical qubits.
        physical_ibm_qubits = ibm_kolkata_ordering[: n_qubits ]
    else:
        raise ValueError(f"Number of qubits {n_qubits} too large.")
    
    return physical_ibm_qubits

In [189]:
def get_layout(n_qubits):
    phys_qubits = get_phys_qubits(n_qubits)
    virt_qubits =[]
    for qubit_i in range(0,n_qubits):
        virt_qubits.append(qubit_i)
    layout =  dict(zip(phys_qubits, virt_qubits))
        
    return layout
        
    

In [190]:
print(get_layout(10))

{0: 0, 1: 1, 4: 2, 7: 3, 10: 4, 12: 5, 15: 6, 18: 7, 21: 8, 23: 9}


In [191]:
def get_computer(n_qubits):
    # Make connectivity graph 
    computer = nx.Graph()

    qubits = [j for j in range(n_qubits)]

    # Split qubits into 2-qubit pairs (assuming a chain connectivity).
    rb_pattern = [[qa, qb] for qa, qb in zip(qubits[0:-1:2], qubits[1::2])]
    if n_qubits % 2 == 1:
        # For an odd number of qubits, append final individual qubit to the RB pattern.
        rb_pattern.append([qubits[-1]])
    # print("Qubit indeces:", qubits)
    # print("RB pattern:", rb_pattern)

    # Assume chain-like connectivity
    computer.add_edges_from([(qa, qb) for qa, qb in zip(qubits[:-1], qubits[1:])])

    # Add reversed edges to computer graph.
    # This is important to represent CNOT gates with target and control reversed.
    computer = nx.to_directed(computer)
    return computer

In [ ]:
n_qubits=10
depth = 3
trials = 3
shots = 10000
num_samples = 100

computer= get_computer(n_qubits)
print(computer)

noisy_backend = GenericBackendV2(25)
ideal_backend = qiskit_aer.AerSimulator()

DiGraph with 10 nodes and 18 edges


In [193]:
def execute(
    circuits: qiskit.QuantumCircuit | list[qiskit.QuantumCircuit],
    backend,
    shots: int,
    correct_bitstring: str,
    verbose: bool,
    ) -> List[float]:
    """Executes the input circuit(s) and returns ⟨A⟩, where A = |correct_bitstring⟩⟨correct_bitstring| for each circuit."""

    if not isinstance(circuits, list):
        circuits = [circuits]
    if verbose:
        # Calculate average number of CNOT gates per circuit.
        print(f"Executing {len(circuits)} circuit(s) on {backend}.")
        print(f"Average cnot count in circuits: {get_avg_cnot_count(circuits)}")

    # Store all circuits to run in list to be returned.
    to_run: list[qiskit.QuantumCircuit] = []

    for circuit in circuits:
        circuit_to_run = circuit.copy()
        circuit_to_run.measure_all()
        to_run.append(
            qiskit.transpile(
                circuit_to_run,
                backend=backend,
                initial_layout= get_phys_qubits(circuit.num_qubits),
                optimization_level=0,  # Otherwise RB circuits are simplified to empty circuits.
            )
        )

    if verbose:
        # Calculate average number of CNOT gates per compiled circuit.
        print(f"Average cnot count in compiled circuits: {get_avg_cnot_count(to_run)}")

    # Run and get counts.
    job = backend.run(
        to_run,
        # Reset qubits to ground state after each sample.
        init_qubits=True,
        shots=shots,
    )

    if len(circuits) == 1:
        return [job.result().get_counts().get(correct_bitstring, 0.0) / shots]
    return [
        count.get(correct_bitstring, 0.0) / shots for count in job.result().get_counts()
    ]

In [194]:
def get_cnot_error(edge: tuple[int, int] = None) -> float:
    cnot_error_prob = 0.01

    # cnot_error_prob = noisy_backend.properties().gate_error("cx", qubits=edge)

    print(f"cnot_error_prob for edge {edge}: {cnot_error_prob}")
    return cnot_error_prob


def get_cnot_representation(edge: tuple[int, int]) -> pec.OperationRepresentation:
    cnot_circuit = cirq.Circuit(
        cirq.CNOT(
            cirq.NamedQubit(f"q_{str(edge[0])}"),
            cirq.NamedQubit(f"q_{str(edge[1])}"),
        )
    )

    rep_exact_prob = 1 - np.sqrt(1 - get_cnot_error(edge))
    return pec.represent_operation_with_local_depolarizing_noise(
        cnot_circuit,
        noise_level=rep_exact_prob,
    )


def get_representations(computer: nx.Graph) -> list[pec.OperationRepresentation]:
    return [get_cnot_representation(edge) for edge in computer.edges]

In [195]:
print(get_phys_qubits(n_qubits))

[0, 1, 4, 7, 10, 12, 15, 18, 21, 23]


In [196]:
local_seed = 10**6 * depth + 10**3 * seed + 3

circuit, correct_bitstring = benchmarks.generate_mirror_circuit(
        nlayers=depth,
        two_qubit_gate_prob=1.0,
        connectivity_graph=computer,
        two_qubit_gate_name="CNOT",
        seed=seed,
        return_type="qiskit",
    )
# Reversed because Qiskit is wrong endian.
correct_bitstring = "".join(map(str, correct_bitstring[::-1]))

(true_value,) = execute(circuit,
    ideal_backend,
    shots,
    correct_bitstring,
    verbose=verbose,
)

(noisy_value,) = execute(circuit,
    noisy_backend,
    shots,
    correct_bitstring,
    verbose=verbose,
)

pec_executor = functools.partial(
        execute,
        backend=noisy_backend,
        shots=shots // num_samples,
        correct_bitstring=correct_bitstring,
        verbose=verbose,
    )
pec_value = pec.execute_with_pec(
        circuit,
        pec_executor,
        representations=get_representations(computer),
        num_samples=num_samples,
        random_state=local_seed,
    )

cnot_error_prob for edge (0, 1): 0.01
cnot_error_prob for edge (1, 0): 0.01
cnot_error_prob for edge (1, 2): 0.01
cnot_error_prob for edge (2, 1): 0.01
cnot_error_prob for edge (2, 3): 0.01
cnot_error_prob for edge (3, 2): 0.01
cnot_error_prob for edge (3, 4): 0.01
cnot_error_prob for edge (4, 3): 0.01
cnot_error_prob for edge (4, 5): 0.01
cnot_error_prob for edge (5, 4): 0.01
cnot_error_prob for edge (5, 6): 0.01
cnot_error_prob for edge (6, 5): 0.01
cnot_error_prob for edge (6, 7): 0.01
cnot_error_prob for edge (7, 6): 0.01
cnot_error_prob for edge (7, 8): 0.01
cnot_error_prob for edge (8, 7): 0.01
cnot_error_prob for edge (8, 9): 0.01
cnot_error_prob for edge (9, 8): 0.01


c:\Users\spira\anaconda3\envs\PHYS522_yml\Lib\site-packages\mitiq\pec\sampling.py:75: UserWarning: No representation found for 

q_0: ───Y───.
  warnings.warn(
c:\Users\spira\anaconda3\envs\PHYS522_yml\Lib\site-packages\mitiq\pec\sampling.py:75: UserWarning: No representation found for 

q_1: ───Ry(0.5π)───.
  warnings.warn(
c:\Users\spira\anaconda3\envs\PHYS522_yml\Lib\site-packages\mitiq\pec\sampling.py:75: UserWarning: No representation found for 

q_2: ───X^-0.5───.
  warnings.warn(
c:\Users\spira\anaconda3\envs\PHYS522_yml\Lib\site-packages\mitiq\pec\sampling.py:75: UserWarning: No representation found for 

q_3: ───X^0.5───.
  warnings.warn(
c:\Users\spira\anaconda3\envs\PHYS522_yml\Lib\site-packages\mitiq\pec\sampling.py:75: UserWarning: No representation found for 

q_4: ───Ry(0.5π)───.
  warnings.warn(
c:\Users\spira\anaconda3\envs\PHYS522_yml\Lib\site-packages\mitiq\pec\sampling.py:75: UserWarning: No representation found for 

q_5: ───Ry(0.5π)───.
  warnings.warn(
c:\Users\s